## Setup

In [ ]:
%pip install vanna

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 KB 271.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 2.8 MB/s eta 0:00:0000:0100:010m
  Preparing metadata (setup.py) ... 

In [2]:
import vanna
from vanna.remote import VannaDefault
import logging


In [3]:
api_key = 'e7c46615470b43d196aa1e2e8eece0a2'

vanna_model_name = 'chatwithquran'
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

In [4]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

logger.info("Starting Vanna setup process")

# # Retrieve secrets from Key Vault
# logger.info("Retrieving credentials from Key Vault")
# db_user = key_vault_client.get_secret(vault_key_username).value
# db_password = key_vault_client.get_secret(vault_key_password).value
# logger.info("Successfully retrieved credentials")

# Connect to MySQL database
logger.info("Connecting to MySQL database")
vn.connect_to_sqlite('qurantranslation')
logger.info("Database connection established")



2024-12-25 18:57:58,614 - INFO - Starting Vanna setup process
2024-12-25 18:57:58,616 - INFO - Connecting to MySQL database
2024-12-25 18:57:58,617 - INFO - Database connection established


## Training
You only need to train once. Do not train again unless you want to add more training data.

In [ ]:
# #     # Fetch schema information
# logger.info("Fetching database schema information")
# df_information_schema = vn.run_sql('SELECT * FROM INFORMATION_SCHEMA;')
# logger.info(f"Retrieved schema information for {len(df_information_schema)} columns")
# #
# #     # Generate training plan
# logger.info("Generating training plan")
# plan = vn.get_training_plan_generic(df_information_schema)
# logger.info("Training plan generated successfully")

# Train with DDL
logger.info("Training with DDL statements")
vn.train(ddl="""
    CREATE TABLE qurantranslation (
        Name TEXT,
        Surah INTEGER,
        Ayat INTEGER,
        Arabic TEXT,
        Translation_Tahir_ul_Qadri TEXT,
        Translation_ArthurJ TEXT, 
        Translation_Marmaduke_Pickthall TEXT,
        Tafaseer_al_Jalalayn TEXT,
        Tafaseer_Tanwir_al_Miqbas TEXT,
        EnglishTitle TEXT,
        ArabicTitle TEXT,
        RomanTitle TEXT,
        NumberOfVerses INTEGER,
        NumberOfRukus TEXT,
        PlaceOfRevelation TEXT,
        PRIMARY KEY (Surah, Ayat)
    );

""")

# Train with documentation
logger.info("Adding business documentation")
with open('Documentation1.txt', 'r') as file:
    # Read all lines and split by comma
    # Open file with UTF-8 encoding
    with open('documentation_list.txt', 'r', encoding='utf-8') as file:
        docs = file.read().split(',')


    for doc in docs:
        # Clean up the documentation string
        doc = doc.strip().strip('"').strip()
        if doc:  # Check if doc is not empty
            # Train the model
            vn.train(documentation=doc)
            print(f"Trained with documentation: {doc[:50]}...")

# logger.info("Documentation training completed")

# Train with example SQL
# logger.info("Training with example SQL queries")
# # Read the file and train the model
# with open('golden_dataset.txt', 'r') as file:
#     for line in file:
#         # Split the line into question and SQL parts
#         question, sql = line.strip().split(',', 1)
#         # Remove the trailing semicolon from SQL
#         sql = sql.rstrip(';')
#         # Train the model
#         vn.train(question=question, sql=sql)

# logger.info("SQL query training completed")
vn.train(sql="SELECT * FROM qurantranslation WHERE Translation_Tahir_ul_Qadri LIKE '%truth%'")
# Get final training data
logger.info("Retrieving final training data")
training_data = vn.get_training_data()
logger.info("Setup complete - Vanna is ready to use")


## Launch the User Interface

In [6]:
# vn.ask("give me all the translations, tafseer of all the ayat that contains misguidance in it")

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn, allow_llm_to_see_data=True)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
None
None
None
None
None
None
None
None
None
None
